### 编者按
- 生成最后模型一共有3份代码：
- 1）生成w2v特征：构建w2v特征，将特征喂给第一期的训练集，项目第二期的训练集和最终的测试集
- 2）stacking 特征代码：主要作用是将项目第一期的训练分布通过嫁接的方式存储到第二期项目
- 3）生成最终结果：主要用于将最后结果预测出最终的结果

#### 特征工程

#### 导入所需的包

In [9]:
import os
import pandas as pd
import lightgbm as lgb
import numpy as np
#从sklearn 评价函数中导入f1-score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

#### 读取数据

In [10]:
#设置根路径
path = './'
w2v_path = './w2v'
# 读取第二期训练集，测试集
train = pd.read_csv('./input/train_2.csv')#第二期训练集
test = pd.read_csv(path + 'input/test_2.csv') # 第二期测试集

# 读取第一期训练集
train_first = pd.read_csv(path + 'input/train_all.csv')

In [11]:
#读取stacking 特征
train_stacking = pd.read_csv(path + '/stack/train.csv')
test_stacking = pd.read_csv(path + '/stack/test.csv')

In [12]:
#将stacking 特征合并
print(len(train), len(test))
train = train.merge(train_stacking, 'left', 'user_id')
test = test.merge(test_stacking, 'left', 'user_id')
print(len(train), len(test))

374655 160566
374655 160566


In [13]:
#将第二期训练集 data_type 为0，第一期训练集设置为1
train['data_type'] = 0
test['data_type'] = 0
train_first['data_type'] = 1

#### 将train,test合并
- 合并主要用于做特征，并将特征合并
- 并且填充空值为0

In [14]:
data = pd.concat([train, test], ignore_index=True).fillna(0)
train_first = train_first.fillna(0)
data['label'] = data.current_service.astype(int)

####  将\\N 代替为0

In [15]:
data = data.replace('\\N', 0)
train_first = train_first.replace('\\N', 0)

#### 将data 中gender 列转成 int格式

In [16]:
data['gender'] = data.gender.astype(int)
train_first['gender'] = train_first.gender.astype(int)

#### 将data service_type 3, 同时设置为4

In [17]:
data.loc[data['service_type'] == 3, 'service_type'] = 4

#### 设置原始特征列

In [18]:
# 设置原始类别特征
origin_cate_feature = ['service_type', 'complaint_level', 'contract_type', 'gender', 'is_mix_service',
                       'is_promise_low_consume',
                       'many_over_bill', 'net_service']

#设置数值特征
origin_num_feature = ['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee',
                      'age', 'contract_time',
                      'former_complaint_fee', 'former_complaint_num',
                      'last_month_traffic', 'local_caller_time', 'local_trafffic_month', 'month_traffic',
                      'online_time', 'pay_num', 'pay_times', 'service1_caller_time', 'service2_caller_time']


#### 原始数值特征转成float型

In [19]:
for i in origin_num_feature:
    data[i] = data[i].astype(float)
    train_first[i] = train_first[i].astype(float)

#### 将w2v特征喂至 data中

In [20]:
w2v_features = []
# 读取w2v 特征
for col in ['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']:
    df = pd.read_csv(w2v_path + '/' + col + '.csv')
    #将读取后的w2v值去重
    df = df.drop_duplicates([col])
    fs = list(df)
    fs.remove(col)
    w2v_features += fs
    print(len(data))
    #将w2v特征merge 进data数据里面
    data = pd.merge(data, df, on=col, how='left')
    print(len(data))
print(w2v_features)

535221
535221
535221
535221
535221
535221
535221
535221
['1_total_feeW0', '1_total_feeW1', '1_total_feeW2', '1_total_feeW3', '1_total_feeW4', '1_total_feeW5', '1_total_feeW6', '1_total_feeW7', '1_total_feeW8', '1_total_feeW9', '2_total_feeW0', '2_total_feeW1', '2_total_feeW2', '2_total_feeW3', '2_total_feeW4', '2_total_feeW5', '2_total_feeW6', '2_total_feeW7', '2_total_feeW8', '2_total_feeW9', '3_total_feeW0', '3_total_feeW1', '3_total_feeW2', '3_total_feeW3', '3_total_feeW4', '3_total_feeW5', '3_total_feeW6', '3_total_feeW7', '3_total_feeW8', '3_total_feeW9', '4_total_feeW0', '4_total_feeW1', '4_total_feeW2', '4_total_feeW3', '4_total_feeW4', '4_total_feeW5', '4_total_feeW6', '4_total_feeW7', '4_total_feeW8', '4_total_feeW9']


#### 新建count 特征列表，用于存储count特征

In [21]:
count_feature_list = []

#### 封装count 特征函数

In [22]:
def feature_count(data, features=[]):
    
    if len(set(features)) != len(features):
        print('equal feature !!!!')
        return data
    #计算count 特征命名
    new_feature = 'count'
    for i in features:
        new_feature += '_' + i.replace('add_', '')
    #尝试删除原有特征名称
    try:
        del data[new_feature]
    except:
        pass
    #临时计算临时的 特征的count特征
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    
    #将临时特征merge 进入data的DataFrame中
    data = data.merge(temp, 'left', on=features)
    #将count特征列表加入新特征
    count_feature_list.append(new_feature)
    return data

#### 计算一系列的count 特征
- 计算 ['1_total_fee','2_total_fee','3_total_fee','4_total_fee','former_complaint_fee','pay_num','contract_time','last_month_traffic','online_time'] 每个列表的count特征
- 计算['service_type','contract_type']与其他类型的组合特征

In [23]:
#计算1-4月份费用的count特征
data = feature_count(data, ['1_total_fee'])
data = feature_count(data, ['2_total_fee'])
data = feature_count(data, ['3_total_fee'])
data = feature_count(data, ['4_total_fee'])
data = feature_count(data, ['former_complaint_fee'])
data = feature_count(data, ['pay_num'])
data = feature_count(data, ['contract_time'])
data = feature_count(data, ['last_month_traffic'])
data = feature_count(data, ['online_time'])
# 计算组合特征的count 特征，举例子：'service_type'+'1_total_fee' 特征
for i in ['service_type', 'contract_type']:
    data = feature_count(data, [i, '1_total_fee'])
    data = feature_count(data, [i, '2_total_fee'])
    data = feature_count(data, [i, '3_total_fee'])
    data = feature_count(data, [i, '4_total_fee'])

    data = feature_count(data, [i, 'former_complaint_fee'])

    data = feature_count(data, [i, 'pay_num'])
    data = feature_count(data, [i, 'contract_time'])
    data = feature_count(data, [i, 'last_month_traffic'])
    data = feature_count(data, [i, 'online_time'])

#### 计算差值特征

In [24]:
#计算同类型的特征的差值（key）
data['diff_total_fee_1'] = data['1_total_fee'] - data['2_total_fee']
data['diff_total_fee_2'] = data['2_total_fee'] - data['3_total_fee']
data['diff_total_fee_3'] = data['3_total_fee'] - data['4_total_fee']
data['pay_num_1_total_fee'] = data['pay_num'] - data['1_total_fee']
data['last_month_traffic_rest'] = data['month_traffic'] - data['last_month_traffic']

In [25]:
data['last_month_traffic_rest'] = data['month_traffic'] - data['last_month_traffic']


#### 计算ratio 特征

In [26]:
#计算电话时长的累加+ratio特征
data['total_caller_time'] = data['service2_caller_time'] + data['service1_caller_time']
data['service2_caller_ratio'] = data['service2_caller_time'] / data['total_caller_time']
data['local_caller_ratio'] = data['local_caller_time'] / data['total_caller_time']

#计算流量的占比和累加特征
data['total_month_traffic'] = data['local_trafffic_month'] + data['month_traffic']
data['month_traffic_ratio'] = data['month_traffic'] / data['total_month_traffic']
data['last_month_traffic_ratio'] = data['last_month_traffic'] / data['total_month_traffic']


#### 猜测套餐金额特征

In [27]:
data['1_total_fee_call_fee'] = data['1_total_fee'] - data['service1_caller_time'] * 0.15
data['1_total_fee_call2_fee'] = data['1_total_fee'] - data['service2_caller_time'] * 0.15
data['1_total_fee_trfc_fee'] = data['1_total_fee'] - (
        data['month_traffic'] - 2 * data['last_month_traffic']) * 0.3


#### 计算4个月 的最大值，最小值，平均值特征

In [28]:
total_fee = []
for i in range(1, 5):
    total_fee.append(str(i) + '_total_fee')
# def get_mean(x1,x2,x3,x4):
#     x = np.mean([x1,x2,x3,x4])
#     return x
# data['total_fee_mean'] = data.apply(lambda row:get_mean(row['0_total_fee'],row['1_total_fee'],row['2_total_fee'],row['3_total_fee']),axis=1)
# data['total_fee_mean'] =data.apply(lambda row:(row['0_total_fee']+row['1_total_fee']+row['2_total_fee']+row['3_total_fee'])/4)
#计算4个月份的最大值，最小值，平均值
data['total_fee_mean'] = data[total_fee].mean(1)
data['total_fee_max'] = data[total_fee].max(1)
data['total_fee_min'] = data[total_fee].min(1)

#### 再次预处理

In [29]:
#金额没有负值，将小于0的金额转为0
data['last_month_traffic_rest'][data['last_month_traffic_rest'] < 0] = 0
#猜测金额对当前月份的金额的占比以判断是否需要变更套餐
data['rest_traffic_ratio'] = (data['last_month_traffic_rest'] * 15 / 1024) / data['1_total_fee']

data.loc[data.service_type == 1, '1_total_fee_trfc_fee'] = None


/Users/yueyang/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### 汇总count特征，ratio,猜测金额以及其余的部分特征：命名为diff_feature_list

In [30]:
# 差值特征
diff_feature_list = ['diff_total_fee_1', 'diff_total_fee_2', 'diff_total_fee_3', 'last_month_traffic_rest',
                     'rest_traffic_ratio',
                     'total_fee_mean', 'total_fee_max', 'total_fee_min', 'total_caller_time', 'service2_caller_ratio',
                     'local_caller_ratio',
                     'total_month_traffic', 'month_traffic_ratio', 'last_month_traffic_ratio', 'pay_num_1_total_fee',
                     '1_total_fee_call_fee', '1_total_fee_call2_fee', '1_total_fee_trfc_fee']


#### 汇总 类别特征和 数值特征
- 类别特征cate_feature
- 数值特征 num_feature

In [31]:
#汇总所有特征，原始类别特征，原始数值特征，count 特征 ，差值特征，以及w2v特征，stacking特征

cate_feature = origin_cate_feature
num_feature = origin_num_feature + count_feature_list + diff_feature_list + w2v_features + ['stacking_' + str(i) for i
                                                                                            in range(11)]

#### 将类别特征转成‘category’类型，数值特征转成float型

In [32]:
# lgb模型可以指定 category 特征（类别特征）
for i in cate_feature:
    data[i] = data[i].astype('category')
# 将数值特征 转成float特征
for i in num_feature:
    data[i] = data[i].astype(float)
#总的特征汇总成类别特征和数值特征
feature = cate_feature + num_feature

#### 打印特征长，并看看有哪些 特征

In [33]:
print(len(feature), feature)

121 ['service_type', 'complaint_level', 'contract_type', 'gender', 'is_mix_service', 'is_promise_low_consume', 'many_over_bill', 'net_service', '1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee', 'age', 'contract_time', 'former_complaint_fee', 'former_complaint_num', 'last_month_traffic', 'local_caller_time', 'local_trafffic_month', 'month_traffic', 'online_time', 'pay_num', 'pay_times', 'service1_caller_time', 'service2_caller_time', 'count_1_total_fee', 'count_2_total_fee', 'count_3_total_fee', 'count_4_total_fee', 'count_former_complaint_fee', 'count_pay_num', 'count_contract_time', 'count_last_month_traffic', 'count_online_time', 'count_service_type_1_total_fee', 'count_service_type_2_total_fee', 'count_service_type_3_total_fee', 'count_service_type_4_total_fee', 'count_service_type_former_complaint_fee', 'count_service_type_pay_num', 'count_service_type_contract_time', 'count_service_type_last_month_traffic', 'count_service_type_online_time', 'count_contract_type_1_total_f

#### 查看service_type 不同的分布

In [34]:
data[data.label != 0]['service_type'].value_counts()

1    248959
4    125696
Name: service_type, dtype: int64

#### 筛选出label 不等于999999的训练集和y

In [35]:
from sklearn.model_selection import train_test_split
datatrainall = data[(data.label != 0)&(data.label != 999999)][feature]
target = data[(data.label != 0)&(data.label != 999999)][['label']]

In [36]:
target

,label
0,90063345
1,90063345
2,90063345
3,90063345
4,90063345
...,...
374650,90063345
374651,90109916
374652,90063345
374653,90109916


#### target labelencoder

In [37]:
from sklearn.preprocessing import LabelEncoder
lbltarget1 = LabelEncoder()
y_ = lbltarget1.fit_transform(target)
y2=lbltarget1.inverse_transform(y_)

/Users/yueyang/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


#### y2lbltarget1.inverse_transform

In [38]:
target.label.value_counts()

90063345    217513
89950167     39307
89950166     26420
90109916     18299
99999828     17164
90155946     13147
99999830     11546
99999826      9279
99999827      8526
89950168      7374
99999825      6078
Name: label, dtype: int64

In [39]:
X_train, X_test, y_train, y_test =train_test_split(datatrainall, y_, test_size=0.3)

#### 创建成lgb特征的数据集格式

In [40]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval  = lgb.Dataset(X_test, y_test, reference=lgb_train)

#### 第一个模型的原生态lgb 的参数

In [41]:
params = {
    'boosting_type': 'gbdt',    # 设置提升类型
    'objective': 'multiclass',  # 目标函数
    'num_leaves': 31,           # 一棵树的叶子节点数
    'learning_rate': 0.03,      # 学习速率
    'num_class':12,             # 类别数
    'feature_fraction': 0.9,    # 建树的特征采样比例
    'bagging_fraction': 0.8,    # 建树的样本采样比例
    'bagging_freq': 5,          # 每k次迭代执行bagging

}



#### 训练label不等于999999的特征

In [42]:
gbm = lgb.train(params, lgb_train, num_boost_round=1000, valid_sets=lgb_eval, early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.101366 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25696
[LightGBM] [Info] Number of data points in the train set: 262257, number of used features: 121


/Users/yueyang/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/yueyang/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score -2.652095
[LightGBM] [Info] Start training from score -2.257979
[LightGBM] [Info] Start training from score -3.924134
[LightGBM] [Info] Start training from score -0.543788
[LightGBM] [Info] Start training from score -3.024186
[LightGBM] [Info] Start training from score -3.340064
[LightGBM] [Info] Start training from score -4.117711
[LightGBM] [Info] Start training from score -3.695525
[LightGBM] [Info] Start training from score -3.791833
[LightGBM] [Info] Start training from score -3.072819
[LightGBM] [Info] Start training from score -3.485518
[LightGBM] [Info] Start training from score -34.538776
[1]	valid_0's multi_logloss: 1.33204
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 1.20409
[3]	valid_0's multi_logloss: 1.10522
[4]	valid_0's multi_logloss: 1.02376
[5]	valid_0's multi_logloss: 0.953806
[6]	valid_0's multi_logloss: 0.893673
[7]	valid_0's multi_logloss: 0.839142
[8]	valid_0's multi_logloss

[192]	valid_0's multi_logloss: 0.114934
[193]	valid_0's multi_logloss: 0.114896
[194]	valid_0's multi_logloss: 0.114862
[195]	valid_0's multi_logloss: 0.114835
[196]	valid_0's multi_logloss: 0.114788
[197]	valid_0's multi_logloss: 0.114743
[198]	valid_0's multi_logloss: 0.114725
[199]	valid_0's multi_logloss: 0.114691
[200]	valid_0's multi_logloss: 0.11466
[201]	valid_0's multi_logloss: 0.114636
[202]	valid_0's multi_logloss: 0.114605
[203]	valid_0's multi_logloss: 0.114593
[204]	valid_0's multi_logloss: 0.114572
[205]	valid_0's multi_logloss: 0.114552
[206]	valid_0's multi_logloss: 0.114527
[207]	valid_0's multi_logloss: 0.1145
[208]	valid_0's multi_logloss: 0.114469
[209]	valid_0's multi_logloss: 0.114438
[210]	valid_0's multi_logloss: 0.114421
[211]	valid_0's multi_logloss: 0.114375
[212]	valid_0's multi_logloss: 0.114353
[213]	valid_0's multi_logloss: 0.114333
[214]	valid_0's multi_logloss: 0.114301
[215]	valid_0's multi_logloss: 0.11427
[216]	valid_0's multi_logloss: 0.114245
[217

[398]	valid_0's multi_logloss: 0.11261
[399]	valid_0's multi_logloss: 0.112603
[400]	valid_0's multi_logloss: 0.112615
[401]	valid_0's multi_logloss: 0.112616
[402]	valid_0's multi_logloss: 0.112612
[403]	valid_0's multi_logloss: 0.11262
[404]	valid_0's multi_logloss: 0.112617
[405]	valid_0's multi_logloss: 0.112626
[406]	valid_0's multi_logloss: 0.112626
[407]	valid_0's multi_logloss: 0.112639
[408]	valid_0's multi_logloss: 0.11264
[409]	valid_0's multi_logloss: 0.112646
[410]	valid_0's multi_logloss: 0.112648
[411]	valid_0's multi_logloss: 0.112656
[412]	valid_0's multi_logloss: 0.112662
[413]	valid_0's multi_logloss: 0.112659
[414]	valid_0's multi_logloss: 0.112665
[415]	valid_0's multi_logloss: 0.112669
[416]	valid_0's multi_logloss: 0.112663
[417]	valid_0's multi_logloss: 0.112664
[418]	valid_0's multi_logloss: 0.112661
[419]	valid_0's multi_logloss: 0.112655
[420]	valid_0's multi_logloss: 0.112649
[421]	valid_0's multi_logloss: 0.112637
[422]	valid_0's multi_logloss: 0.112629
[42

#### 筛选出service_type =1的模型来进行预测

In [43]:
result_type1 = pd.DataFrame()
result_type1['user_id'] = data[(data.label == 0) & (data.service_type == 1)]['user_id']
result_type1['predict'] = lbltarget1.inverse_transform(np.argmax(gbm.predict(data[(data.label == 0) & (data.service_type == 1)][feature]),axis=1))
print(len(result_type1))

106801


In [44]:
result_type1.predict.value_counts()

90063345    93222
90109916     7973
90155946     5606
Name: predict, dtype: int64

#### 筛选出service_type =4 的训练集，并将index reset

In [45]:
X = data[(data.label != 0) & (data.label != 999999)&(data.service_type == 4)][feature].reset_index(drop=True)
y = data[(data.label != 0) & (data.label != 999999)&(data.service_type == 4)][['label']].reset_index(drop=True)

#### 查看 label的分布

In [46]:
y['label'].value_counts()

89950167    39307
89950166    26420
99999828    17164
99999830    11546
99999826     9279
99999827     8526
89950168     7374
99999825     6078
Name: label, dtype: int64

In [47]:
X.shape

(125694, 121)

#### 将第二份数据的label 进行labelEncoder

In [48]:
from sklearn.preprocessing import LabelEncoder
lbltarget2 = LabelEncoder()
yservice4 = lbltarget2.fit_transform(y)
# y2=lbltarget1.inverse_transform(y_)

/Users/yueyang/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


#### 第二个模型的原生态lgb的参数

In [49]:
params2 = {
    'boosting_type': 'gbdt',    # 设置提升类型
    'objective': 'multiclass',  # 目标函数
    'num_leaves': 31,           # 一棵树的叶子节点数
    'learning_rate': 0.03,      # 学习速率
    'num_class':8,             # 类别数
    'feature_fraction': 0.9,    # 建树的特征采样比例
    'bagging_fraction': 0.8,    # 建树的样本采样比例
    'bagging_freq': 5,          # 每k次迭代执行bagging
    'metric': 'multi_error',

}

#### 因为接下来需要跑一个5折模型，所以需要校验线下的话，需要提前设置一个8分类的oof:out of fold

In [50]:
oof2= np.zeros((125694,8)) 

#### 初始化测试集8分类概率的初始化

In [51]:
predictprob=np.zeros((len(data[(data.label == 0) & (data.service_type != 1)][feature]),8)) 

#### 设置matric

In [52]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    preds = preds.reshape(-1, 8)
    preds = preds.argmax(axis = 1)
    f_score = f1_score(preds, labels, average = 'micro')
    return 'f1_score', f_score, False

In [53]:
def self_metric(labels, preds):

    preds = preds.get_label()

    score = 1-np.mean(np.abs(labels-preds)/(np.abs(labels)+np.abs(preds)+0.1))

    return 'self_metric', score, True

#### 训练五折模型
    gbmskf.best_iteration 最优步数
    loc索引
    num_boost_round 步数
    valid_sets 验证集
    early_stoping_rounds 早停步数


In [54]:
# # 创建成lgb特征的数据集格式
# lgb_train = lgb.Dataset(X, yservice4)
# lgb_eval  = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [55]:
skf = StratifiedKFold(n_splits=5, random_state=20181, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(index)
    train_x, vali_x, train_y, vali_y = X.loc[train_index], X.loc[test_index], yservice4[train_index], yservice4[test_index]
    
    lgb_train = lgb.Dataset(train_x, train_y)
    lgb_eval  = lgb.Dataset(vali_x, vali_y, reference=lgb_train)
    gbmskf = lgb.train(params2, lgb_train, num_boost_round=1000, valid_sets=lgb_eval, early_stopping_rounds=100)
    oof2[test_index]=gbmskf.predict(vali_x,num_iteration=gbmskf.best_iteration)
    
    predictprob += gbmskf.predict(data[(data.label == 0) & (data.service_type != 1)][feature],num_iteration=gbmskf.best_iteration,verbose_eval = 1)/5
    
    

0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050520 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25107
[LightGBM] [Info] Number of data points in the train set: 100555, number of used features: 120
[LightGBM] [Info] Start training from score -1.559727
[LightGBM] [Info] Start training from score -1.162433
[LightGBM] [Info] Start training from score -2.835922
[LightGBM] [Info] Start training from score -3.029049
[LightGBM] [Info] Start training from score -2.606122
[LightGBM] [Info] Start training from score -2.690845
[LightGBM] [Info] Start training from score -1.991049
[LightGBM] [Info] Start training from score -2.387488


/Users/yueyang/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/yueyang/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's multi_error: 0.687299
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_error: 0.687299
[3]	valid_0's multi_error: 0.632404
[4]	valid_0's multi_error: 0.519949
[5]	valid_0's multi_error: 0.426986
[6]	valid_0's multi_error: 0.391583
[7]	valid_0's multi_error: 0.220932
[8]	valid_0's multi_error: 0.193604
[9]	valid_0's multi_error: 0.177811
[10]	valid_0's multi_error: 0.162178
[11]	valid_0's multi_error: 0.15466
[12]	valid_0's multi_error: 0.149011
[13]	valid_0's multi_error: 0.145392
[14]	valid_0's multi_error: 0.142607
[15]	valid_0's multi_error: 0.140379
[16]	valid_0's multi_error: 0.138788
[17]	valid_0's multi_error: 0.137436
[18]	valid_0's multi_error: 0.137038
[19]	valid_0's multi_error: 0.136242
[20]	valid_0's multi_error: 0.136203
[21]	valid_0's multi_error: 0.135964
[22]	valid_0's multi_error: 0.135487
[23]	valid_0's multi_error: 0.135208
[24]	valid_0's multi_error: 0.13493
[25]	valid_0's multi_error: 0.134651
[26]	valid_0's multi_err

[220]	valid_0's multi_error: 0.121962
[221]	valid_0's multi_error: 0.121962
[222]	valid_0's multi_error: 0.121882
[223]	valid_0's multi_error: 0.121882
[224]	valid_0's multi_error: 0.121803
[225]	valid_0's multi_error: 0.121723
[226]	valid_0's multi_error: 0.121644
[227]	valid_0's multi_error: 0.121485
[228]	valid_0's multi_error: 0.121524
[229]	valid_0's multi_error: 0.121405
[230]	valid_0's multi_error: 0.121445
[231]	valid_0's multi_error: 0.121405
[232]	valid_0's multi_error: 0.121485
[233]	valid_0's multi_error: 0.121445
[234]	valid_0's multi_error: 0.121365
[235]	valid_0's multi_error: 0.121524
[236]	valid_0's multi_error: 0.121485
[237]	valid_0's multi_error: 0.121564
[238]	valid_0's multi_error: 0.121683
[239]	valid_0's multi_error: 0.121763
[240]	valid_0's multi_error: 0.121763
[241]	valid_0's multi_error: 0.121683
[242]	valid_0's multi_error: 0.121604
[243]	valid_0's multi_error: 0.121445
[244]	valid_0's multi_error: 0.121445
[245]	valid_0's multi_error: 0.121365
[246]	valid_

[437]	valid_0's multi_error: 0.11512
[438]	valid_0's multi_error: 0.115001
[439]	valid_0's multi_error: 0.11504
[440]	valid_0's multi_error: 0.11508
[441]	valid_0's multi_error: 0.115239
[442]	valid_0's multi_error: 0.115239
[443]	valid_0's multi_error: 0.115199
[444]	valid_0's multi_error: 0.115319
[445]	valid_0's multi_error: 0.115398
[446]	valid_0's multi_error: 0.11516
[447]	valid_0's multi_error: 0.115199
[448]	valid_0's multi_error: 0.11508
[449]	valid_0's multi_error: 0.114802
[450]	valid_0's multi_error: 0.114682
[451]	valid_0's multi_error: 0.114722
[452]	valid_0's multi_error: 0.114563
[453]	valid_0's multi_error: 0.114563
[454]	valid_0's multi_error: 0.114603
[455]	valid_0's multi_error: 0.114682
[456]	valid_0's multi_error: 0.114722
[457]	valid_0's multi_error: 0.114643
[458]	valid_0's multi_error: 0.114523
[459]	valid_0's multi_error: 0.114523
[460]	valid_0's multi_error: 0.114483
[461]	valid_0's multi_error: 0.114523
[462]	valid_0's multi_error: 0.114603
[463]	valid_0's m

...

[1]	valid_0's multi_error: 0.687286
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_error: 0.687286
[3]	valid_0's multi_error: 0.637879
[4]	valid_0's multi_error: 0.523351
[5]	valid_0's multi_error: 0.429231
[6]	valid_0's multi_error: 0.390644
[7]	valid_0's multi_error: 0.224998
[8]	valid_0's multi_error: 0.200692
[9]	valid_0's multi_error: 0.183905
[10]	valid_0's multi_error: 0.167038
[11]	valid_0's multi_error: 0.160753
[12]	valid_0's multi_error: 0.155024
[13]	valid_0's multi_error: 0.151046
[14]	valid_0's multi_error: 0.147705
[15]	valid_0's multi_error: 0.144761
[16]	valid_0's multi_error: 0.143209
[17]	valid_0's multi_error: 0.14126
[18]	valid_0's multi_error: 0.139947
[19]	valid_0's multi_error: 0.139231
[20]	valid_0's multi_error: 0.138515
[21]	valid_0's multi_error: 0.138237
[22]	valid_0's multi_error: 0.137481
[23]	valid_0's multi_error: 0.137203
[24]	valid_0's multi_error: 0.136646
[25]	valid_0's multi_error: 0.136248
[26]	valid_0's multi_er

[220]	valid_0's multi_error: 0.120972
[221]	valid_0's multi_error: 0.120893
[222]	valid_0's multi_error: 0.121012
[223]	valid_0's multi_error: 0.121092
[224]	valid_0's multi_error: 0.12133
[225]	valid_0's multi_error: 0.121489
[226]	valid_0's multi_error: 0.121768
[227]	valid_0's multi_error: 0.121489
[228]	valid_0's multi_error: 0.12137
[229]	valid_0's multi_error: 0.121251
[230]	valid_0's multi_error: 0.12129
[231]	valid_0's multi_error: 0.121171
[232]	valid_0's multi_error: 0.12133
[233]	valid_0's multi_error: 0.12129
[234]	valid_0's multi_error: 0.121171
[235]	valid_0's multi_error: 0.120932
[236]	valid_0's multi_error: 0.120853
[237]	valid_0's multi_error: 0.121012
[238]	valid_0's multi_error: 0.120813
[239]	valid_0's multi_error: 0.120734
[240]	valid_0's multi_error: 0.120574
[241]	valid_0's multi_error: 0.120495
[242]	valid_0's multi_error: 0.120336
[243]	valid_0's multi_error: 0.120137
[244]	valid_0's multi_error: 0.120177
[245]	valid_0's multi_error: 0.120256
[246]	valid_0's m

[438]	valid_0's multi_error: 0.118188
[439]	valid_0's multi_error: 0.118227
[440]	valid_0's multi_error: 0.118227
[441]	valid_0's multi_error: 0.118307
[442]	valid_0's multi_error: 0.118307
[443]	valid_0's multi_error: 0.118267
[444]	valid_0's multi_error: 0.118347
[445]	valid_0's multi_error: 0.118267
[446]	valid_0's multi_error: 0.118188
[447]	valid_0's multi_error: 0.118068
[448]	valid_0's multi_error: 0.117989
[449]	valid_0's multi_error: 0.117989
[450]	valid_0's multi_error: 0.118068
[451]	valid_0's multi_error: 0.118108
[452]	valid_0's multi_error: 0.118028
[453]	valid_0's multi_error: 0.118227
[454]	valid_0's multi_error: 0.118188
[455]	valid_0's multi_error: 0.118267
[456]	valid_0's multi_error: 0.118148
[457]	valid_0's multi_error: 0.118068
[458]	valid_0's multi_error: 0.117909
[459]	valid_0's multi_error: 0.117949
[460]	valid_0's multi_error: 0.118068
[461]	valid_0's multi_error: 0.118267
[462]	valid_0's multi_error: 0.118148
[463]	valid_0's multi_error: 0.118267
[464]	valid_

[655]	valid_0's multi_error: 0.114448
[656]	valid_0's multi_error: 0.114329
[657]	valid_0's multi_error: 0.114329
[658]	valid_0's multi_error: 0.114369
[659]	valid_0's multi_error: 0.114528
[660]	valid_0's multi_error: 0.114488
[661]	valid_0's multi_error: 0.114408
[662]	valid_0's multi_error: 0.114528
[663]	valid_0's multi_error: 0.114647
[664]	valid_0's multi_error: 0.114727
[665]	valid_0's multi_error: 0.114727
[666]	valid_0's multi_error: 0.115005
[667]	valid_0's multi_error: 0.115125
[668]	valid_0's multi_error: 0.115204
[669]	valid_0's multi_error: 0.114965
[670]	valid_0's multi_error: 0.115164
[671]	valid_0's multi_error: 0.115204
[672]	valid_0's multi_error: 0.115125
[673]	valid_0's multi_error: 0.115085
[674]	valid_0's multi_error: 0.115125
[675]	valid_0's multi_error: 0.115005
[676]	valid_0's multi_error: 0.114926
[677]	valid_0's multi_error: 0.114926
[678]	valid_0's multi_error: 0.114965
[679]	valid_0's multi_error: 0.114965
[680]	valid_0's multi_error: 0.114687
[681]	valid_

[872]	valid_0's multi_error: 0.113573
[873]	valid_0's multi_error: 0.113653
[874]	valid_0's multi_error: 0.113653
[875]	valid_0's multi_error: 0.113533
[876]	valid_0's multi_error: 0.113573
[877]	valid_0's multi_error: 0.113533
[878]	valid_0's multi_error: 0.113374
[879]	valid_0's multi_error: 0.113414
[880]	valid_0's multi_error: 0.113374
[881]	valid_0's multi_error: 0.113374
[882]	valid_0's multi_error: 0.113613
[883]	valid_0's multi_error: 0.113692
[884]	valid_0's multi_error: 0.113573
[885]	valid_0's multi_error: 0.113692
[886]	valid_0's multi_error: 0.113812
[887]	valid_0's multi_error: 0.113772
[888]	valid_0's multi_error: 0.113613
[889]	valid_0's multi_error: 0.113891
[890]	valid_0's multi_error: 0.113812
[891]	valid_0's multi_error: 0.113812
[892]	valid_0's multi_error: 0.113692
[893]	valid_0's multi_error: 0.113613
[894]	valid_0's multi_error: 0.113573
[895]	valid_0's multi_error: 0.113692
[896]	valid_0's multi_error: 0.113533
[897]	valid_0's multi_error: 0.113494
[898]	valid_

In [56]:
predictprob[0]

array([2.12038490e-01, 3.46499244e-04, 7.10177657e-07, 1.27825335e-07,
       7.52907274e-06, 7.05973946e-04, 3.59189945e-04, 7.86541480e-01])

#### 将预测到的类别进行反转

In [57]:
ooflabel = lbltarget2.inverse_transform(np.argmax((oof2),axis=1))

#### 查看线下的f1 score macro

In [58]:
from sklearn.metrics import f1_score

score = f1_score(y_true=y, y_pred=ooflabel, average='macro')
print(score)

0.8742441362904628


#### 生成最后的结果

In [59]:
result = pd.DataFrame()
result['user_id'] = data[(data.label == 0) & (data.service_type != 1)]['user_id']
result['predict'] = lbltarget2.inverse_transform(np.argmax((predictprob),axis=1))

result.loc[result['user_id'] == '4VNcD6kE0sjnAvFX', 'predict'] = 999999
# 合并第一个模型的結果
result = result.append(result_type1)
result.predict.value_counts()

90063345    93222
89950167    16845
89950166    12656
90109916     7973
99999828     7328
90155946     5606
99999826     4137
99999827     3611
99999830     3409
89950168     3094
99999825     2684
999999          1
Name: predict, dtype: int64

In [60]:
predictprob.shape

(53765, 8)

In [61]:
result.shape

(160566, 2)

#### 生成最后的结果文件

In [62]:

result[['user_id', 'predict']].to_csv(
    path + '/sub.csv', index=False)